# Amazon EDA

Frage: Was sind interessante Eigenschaften der Bücher in den Amazon Top 50 Bestselling Books 2009 - 2019?

In [1]:
# Importe
import pandas as pd


In [2]:
# Einlesen

df = pd.read_csv("../data/bestsellers with categories.csv")
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


In [4]:
# Gibt es exakte Duplikate?
df.duplicated().sum()

np.int64(0)

In [5]:
# Gibt es doppelte Namen?
df.duplicated(subset="Name").sum()

np.int64(199)

In [6]:
# Es gibt 3 Bücher, die in einem Jahr doppelt vorkommen
df.duplicated(subset=["Name", "Author", "Year"]).sum()

np.int64(3)

In [7]:
# Wir schauen uns an, ob es Namen/ Autoren gibt, die verschiedene Schreibweisen haben 

# Spalte Name zu Lowercase
df["name_code"] = df["Name"].str.lower().str.replace(r"[^a-z0-9]", "", regex=True)

In [8]:
# Ein Buch scheint bis auf die Schreibweise gleich gewesen zu sein

df.duplicated(subset=["name_code"]).sum()

np.int64(200)

In [9]:
# Welche Bücher kamen in einem Jahr Doppelt vor?
df[df.duplicated(subset=["Name", "Author", "Year"], keep=False)]

,Name,Author,User Rating,Reviews,Price,Year,Genre,name_code
367,The Fault in Our Stars,John Green,4.7,50482,7,2014,Fiction,thefaultinourstars
368,The Fault in Our Stars,John Green,4.7,50482,13,2014,Fiction,thefaultinourstars
404,The Help,Kathryn Stockett,4.8,13871,8,2011,Fiction,thehelp
405,The Help,Kathryn Stockett,4.8,13871,7,2011,Fiction,thehelp
518,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,13,2014,Non Fiction,unbrokenaworldwariistoryofsurvivalresiliencean...
519,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,4.8,29673,16,2014,Non Fiction,unbrokenaworldwariistoryofsurvivalresiliencean...


In [10]:
df_unique_names_per_year = df.groupby(["Name", "Author", "Year"], as_index=False).mean(numeric_only=True)
df_unique_names_per_year

,Name,Author,Year,User Rating,Reviews,Price
0,10-Day Green Smoothie Cleanse,JJ Smith,2016,4.7,17350.0,8.0
1,11/22/63: A Novel,Stephen King,2011,4.6,2052.0,22.0
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,2018,4.7,18979.0,15.0
3,1984 (Signet Classics),George Orwell,2017,4.7,21424.0,6.0
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,2019,4.8,7665.0,12.0
...,...,...,...,...,...,...
542,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,2019,4.9,9413.0,8.0
543,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,2016,4.7,14331.0,8.0
544,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,2017,4.7,14331.0,8.0
545,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,2018,4.7,14331.0,8.0


In [11]:
# Es scheint ein weiteres Buch zu geben, bei dem der Name nahezu gleich ist. 
# Vielleicht wäre das sogar die bessere Query, um Duplikate zu finden
df_unique_names_per_year[df_unique_names_per_year.duplicated(
    subset=["Author", "Year", "User Rating", "Reviews"], keep=False)]

,Name,Author,Year,User Rating,Reviews,Price
95,Eclipse (Twilight Sagas),Stephenie Meyer,2009,4.7,5505.0,7.0
96,Eclipse (Twilight),Stephenie Meyer,2009,4.7,5505.0,18.0


In [12]:
# Wir Gruppieren und aggregieren Danach die Spalte Price

df_unique_names_per_year = df.groupby(
    ["Author", "Year", "User Rating", "Reviews"], as_index=False).agg({"Name":"first", "Price":"mean", "Genre":"first"})
df_unique_names_per_year

,Author,Year,User Rating,Reviews,Name,Price,Genre
0,Abraham Verghese,2010,4.6,4866,Cutting for Stone,11.0,Fiction
1,Abraham Verghese,2011,4.6,4866,Cutting for Stone,11.0,Fiction
2,Adam Gasiewski,2017,4.4,3113,Milk and Vine: Inspirational Quotes From Class...,6.0,Non Fiction
3,Adam Mansbach,2011,4.8,9568,Go the F**k to Sleep,9.0,Fiction
4,Adir Levy,2019,4.8,8170,What Should Danny Do? (The Power to Choose Ser...,13.0,Fiction
...,...,...,...,...,...,...,...
541,Wizards RPG Team,2017,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
542,Wizards RPG Team,2018,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
543,Wizards RPG Team,2019,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
544,Zhi Gang Sha,2009,4.6,37,Divine Soul Mind Body Healing and Transmission...,6.0,Non Fiction


In [13]:
df_unique_names_per_year.describe()

,Year,User Rating,Reviews,Price
count,546.000000,546.000000,546.000000,546.000000
mean,2014.014652,4.617399,11858.560440,13.114469
std,3.166882,0.227490,11629.168907,10.871273
min,2009.000000,3.300000,37.000000,0.000000
25%,2011.000000,4.500000,4023.500000,7.000000
50%,2014.000000,4.700000,8535.500000,11.000000
75%,2017.000000,4.800000,17030.500000,16.000000
max,2019.000000,4.900000,87841.000000,105.000000


In [14]:
# Welche Bücher haben den Preis 0
df_unique_names_per_year[df_unique_names_per_year["Price"] == 0]

,Author,Year,User Rating,Reviews,Name,Price,Genre
9,Alice Schertle,2014,4.9,1884,Little Blue Truck,0.0,Fiction
118,Delegates of the Constitutional,2016,4.8,2774,The Constitution of the United States,0.0,Non Fiction
225,Harper Lee,2013,4.8,26234,To Kill a Mockingbird,0.0,Fiction
226,Harper Lee,2014,4.8,26234,To Kill a Mockingbird,0.0,Fiction
228,Harper Lee,2015,4.8,26234,To Kill a Mockingbird,0.0,Fiction
229,Harper Lee,2016,4.8,26234,To Kill a Mockingbird,0.0,Fiction
259,Jeff Kinney,2011,4.8,4505,"Cabin Fever (Diary of a Wimpy Kid, Book 6)",0.0,Fiction
261,Jeff Kinney,2013,4.8,6812,"Diary of a Wimpy Kid: Hard Luck, Book 8",0.0,Fiction
265,Jeff Kinney,2017,4.8,5836,The Getaway,0.0,Fiction
387,RH Disney,2014,4.6,978,JOURNEY TO THE ICE P,0.0,Fiction


In [15]:
free_books = df_unique_names_per_year[df_unique_names_per_year["Price"] == 0].drop(["Price", "Year"], axis=1)
free_books

,Author,User Rating,Reviews,Name,Genre
9,Alice Schertle,4.9,1884,Little Blue Truck,Fiction
118,Delegates of the Constitutional,4.8,2774,The Constitution of the United States,Non Fiction
225,Harper Lee,4.8,26234,To Kill a Mockingbird,Fiction
226,Harper Lee,4.8,26234,To Kill a Mockingbird,Fiction
228,Harper Lee,4.8,26234,To Kill a Mockingbird,Fiction
229,Harper Lee,4.8,26234,To Kill a Mockingbird,Fiction
259,Jeff Kinney,4.8,4505,"Cabin Fever (Diary of a Wimpy Kid, Book 6)",Fiction
261,Jeff Kinney,4.8,6812,"Diary of a Wimpy Kid: Hard Luck, Book 8",Fiction
265,Jeff Kinney,4.8,5836,The Getaway,Fiction
387,RH Disney,4.6,978,JOURNEY TO THE ICE P,Fiction


In [16]:
# Finde alle Bücher, die irgendwann mal kostenlos waren

df_unique_names_per_year.merge(free_books,  how="right").drop_duplicates()

,Author,Year,User Rating,Reviews,Name,Price,Genre
0,Alice Schertle,2014,4.9,1884,Little Blue Truck,0.0,Fiction
1,Delegates of the Constitutional,2016,4.8,2774,The Constitution of the United States,0.0,Non Fiction
2,Harper Lee,2013,4.8,26234,To Kill a Mockingbird,0.0,Fiction
3,Harper Lee,2014,4.8,26234,To Kill a Mockingbird,0.0,Fiction
4,Harper Lee,2015,4.8,26234,To Kill a Mockingbird,0.0,Fiction
5,Harper Lee,2016,4.8,26234,To Kill a Mockingbird,0.0,Fiction
6,Harper Lee,2019,4.8,26234,To Kill a Mockingbird,7.0,Fiction
22,Jeff Kinney,2011,4.8,4505,"Cabin Fever (Diary of a Wimpy Kid, Book 6)",0.0,Fiction
23,Jeff Kinney,2013,4.8,6812,"Diary of a Wimpy Kid: Hard Luck, Book 8",0.0,Fiction
24,Jeff Kinney,2017,4.8,5836,The Getaway,0.0,Fiction


In [17]:
# Alternative, um alle Namen von Büchern zu finden, die irgendwann mal kostenlos sind

df_unique_names_per_year[df_unique_names_per_year["Name"].isin(free_books["Name"])]

,Author,Year,User Rating,Reviews,Name,Price,Genre
9,Alice Schertle,2014,4.9,1884,Little Blue Truck,0.0,Fiction
118,Delegates of the Constitutional,2016,4.8,2774,The Constitution of the United States,0.0,Non Fiction
225,Harper Lee,2013,4.8,26234,To Kill a Mockingbird,0.0,Fiction
226,Harper Lee,2014,4.8,26234,To Kill a Mockingbird,0.0,Fiction
228,Harper Lee,2015,4.8,26234,To Kill a Mockingbird,0.0,Fiction
229,Harper Lee,2016,4.8,26234,To Kill a Mockingbird,0.0,Fiction
230,Harper Lee,2019,4.8,26234,To Kill a Mockingbird,7.0,Fiction
259,Jeff Kinney,2011,4.8,4505,"Cabin Fever (Diary of a Wimpy Kid, Book 6)",0.0,Fiction
261,Jeff Kinney,2013,4.8,6812,"Diary of a Wimpy Kid: Hard Luck, Book 8",0.0,Fiction
265,Jeff Kinney,2017,4.8,5836,The Getaway,0.0,Fiction


In [ ]:
# Wir passen den Preis von To Kill a Mockingbird an (auf den Preis, welchen wir in der Tabelle ablesen). 
# Die restlichen Bücher mit Preis 0 entfernen wir
# Wir brauchen hier das .loc, da die Änderung sonst nirgendwo gespeichert wird
df_unique_names_per_year.loc[df_unique_names_per_year["Name"]
                         == "To Kill a Mockingbird", "Price"] = 7
df_unique_names_per_year[df_unique_names_per_year["Name"]
                         == "To Kill a Mockingbird"]

,Author,Year,User Rating,Reviews,Name,Price,Genre
225,Harper Lee,2013,4.8,26234,To Kill a Mockingbird,7.0,Fiction
226,Harper Lee,2014,4.8,26234,To Kill a Mockingbird,7.0,Fiction
228,Harper Lee,2015,4.8,26234,To Kill a Mockingbird,7.0,Fiction
229,Harper Lee,2016,4.8,26234,To Kill a Mockingbird,7.0,Fiction
230,Harper Lee,2019,4.8,26234,To Kill a Mockingbird,7.0,Fiction


In [28]:
# Entferne die restlichen Bücher mit Preis 0

df_unique_names_per_year_price_gt_0 = df_unique_names_per_year[df_unique_names_per_year["Price"] != 0]
df_unique_names_per_year_price_gt_0

,Author,Year,User Rating,Reviews,Name,Price,Genre
0,Abraham Verghese,2010,4.6,4866,Cutting for Stone,11.0,Fiction
1,Abraham Verghese,2011,4.6,4866,Cutting for Stone,11.0,Fiction
2,Adam Gasiewski,2017,4.4,3113,Milk and Vine: Inspirational Quotes From Class...,6.0,Non Fiction
3,Adam Mansbach,2011,4.8,9568,Go the F**k to Sleep,9.0,Fiction
4,Adir Levy,2019,4.8,8170,What Should Danny Do? (The Power to Choose Ser...,13.0,Fiction
...,...,...,...,...,...,...,...
541,Wizards RPG Team,2017,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
542,Wizards RPG Team,2018,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
543,Wizards RPG Team,2019,4.8,16990,Player's Handbook (Dungeons & Dragons),27.0,Fiction
544,Zhi Gang Sha,2009,4.6,37,Divine Soul Mind Body Healing and Transmission...,6.0,Non Fiction


In [30]:
df_unique_names_per_year_price_gt_0.to_csv("../data/cleaned_data_prices.csv")